In [397]:
# Monte Carlo valuation of European Barrier Option under local volatility and LIBOR Forward Market Model 

# Barrier Option type is knock-out option/up and out 
# This option ceases to exist when price exceeds a barrier level L
# The value of call option goes to 0 when S_t > L between 0 and T(maturity)
#1. Pricing of Barrier option under no default risk
#2. Pricing of Barrier option under defualt risk

In [398]:
# Import modules
import random
import numpy as np
from scipy.stats import norm
from scipy.stats import ncx2 
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
np.random.seed(0)

In [399]:
# 1. - Parameters - Initial Values, Market and modelling values and dynamics

# Dynamics of market, share and European-up-and-out call option
S_0                       = 100                                    # inital price of share price
K                         = 100                                    # strike price -option struck at the money
L                         = 150                                    # up and out barrier level
T                         = 1                                      # time to maturity of option 1 year
r                         = 0.08                                   # continously compounded risk free rate
sigma_share               = 0.3                                    # the initial/known volatility of share 


# Counterparty firms dynamics

V_0                       = 200                                    # initial firm value
D                         = 175                                    # firm debt due in 1 year
sigma_firm                = 0.3                                    # the initial/known volatility of firm value 
r_r                       = 0.25                                   # the recovery rate with counterparty

# Correlations share and firm 
corr                      = 0.2                                    # correlation between firm value and share price
corr_matrix               = np.array([[1,corr],[corr,1]])          # correlation matrix stock and firm

# Constant Elasticity Volatility Model
gamma                     = 0.75

# Zero Coupon Bond Market Prices
ZCB_bond_prices         = np.array([100, 99.38,98.76,98.15,97.54,96.94,96.34,95.74,95.16,94.57,93.99,93.42,92.85])

# Monte Carlo Simulation dynamics
num_simulations           = 100000                                 #  number simulations
np.random.seed(0)
dT                        = 1/12                                   # monthly time period
paths                     = int(1/dT)                              # price observation points between (0,T]
t                         = np.linspace(1,paths+1,paths+1)         # monthly time period paths
T                         = t[1:]-t[0:-1]                          # (t_i+1 - t_i)

In [400]:
# 2 - Functions for share price path
# Share prices paths follow Geometric Brownian motion of the form below so we can simulate next share price in price path using
  # S_t_i+1 = S_t_i*exp((r_t_i - sigma_share(t_i,t_i+1)**2/2)*(t_i+1-t_i) + sigma_share(t_i,t_i+1)*sqrt(t_i+1-t_i)*Z
  # where Z is standard normal distribution Z~N(0,1)

def next_share_price(S_curr,r_curr, sigma_share_curr, Z_curr,T):
    """  
    takes current stock price, current stock volatility, current interest rate, Z random aspect..
    ..returns next projected/next stock price based on initial stock price on geometric brownian motion 
    """
    return S_curr*(np.exp((r_curr-sigma_share_curr**2/2)*T + sigma_share_curr*np.sqrt(T)*Z_curr))

In [401]:
# 3 - Functions for firm value paths
# Counterparty value paths follow Geometric Brownian motion of the form so we can simulate final or intermediate firm value 
  # V_t_i+1 = V_t_i*exp((r_t_i - sigma_firm(t_i,t_i+1)**2/2)*(t_i+1-t_i) + sigma_firm(t_i,t_i+1)*sqrt(t_i+1-t_i)*Z
  # where Z is standard normal distribution Z~N(0,1)
  # Assume defualt only occurs at maturity 
    
def next_firm_value(V_curr,r_curr, sigma_firm_curr, Z_curr,T):
    """  
    takes current firm value, current firm volatility, current interest rate, Z random aspect..
    ..returns next projected/next firm value based on initial stock price on geometric brownian motion 
    """
    return V_curr*(np.exp((r_curr-sigma_firm_curr**2/2)*T + sigma_firm_curr*np.sqrt(T)*Z_curr))

In [402]:
# 4 - Functions for local volatility share price
# Local volatility model for stock has function 
  # sigma_share(t_i,t_i+1)  = sigma_share*S_t_i**(gamma-1)  where sigma_share = 0.3
    
def next_share_volatility(sigma_share_curr,S_curr,gamma):
    """  
    takes initial stock volatility, current share price, gamma
    ..returns next projected/next stock volatility
    """
    return sigma_share_curr*S_curr**(gamma-1)

In [403]:
# 5 - Functions for local volatility firm value
# Local volatility model for counterparty has function 
  # sigma_firm(t_i,t_i+1)  = sigma_firm*S_t_i**(gamma-1)    where sigma_firm  = 0.3
    
def next_firm_volatility(sigma_firm_curr,V_curr,gamma):
    """  
    takes initial firm volatility, current firm price, gamma
    ..returns next projected/next firm volatility
    """
    return sigma_firm_curr*V_curr**(gamma-1)

In [404]:
# 6 - Functions discounted payoff European barrier option

def discounted_option_payoff(share_paths,r_paths,K,L,T_paths):   
    """
    takes paths of share (monthly samples) over life of share 
    returns 0 if share price rises above L(barrier) during life of option
    returns normall discounted call payoff at maturity if share price never exceed barrier value
    """  
    discounted_payoffs = [None]*num_simulations
   
    for i in range(num_simulations):
        if(np.any(share_paths[i] > L)):
            discounted_payoffs[i] = 0
        else:
            discounted_payoffs[i] = np.maximum((share_paths[i][-1])-K,0)*np.exp(-(r_paths[i][-1])*T[0])
    return discounted_payoffs   

In [419]:
# 7 - Credit Valuation Adjustment Functions based on Merton Model
def credit_value_adjustment(r_paths,r_r,terminal_value_firm,D,call_value):
    """
    takes terminal interest rate, recovery rate, terminal firm value, firm debt due at time T, call_values
    """
    credit_value_adjustments = [None]*num_simulations
    
    for i in range(num_simulations):
        credit_value_adjustments[i] = np.exp(-(r_paths[i][-1])*T[0])*(1-r_r)*((terminal_value_firm[i][-1]) < D)*call_value   
   
    return credit_value_adjustments

In [420]:
# This needs to be corrected and revised 
#...............................................
# Need to do some calibration here, do we need to use predictor-corrector model
# 8 - Functions interest rate modelling (makes use of LIBOR Forward Market Model)
# LIBOR Rates make use of LIBOR forward market model 
  # exp(r_t_i*(t_i+1-t_i)) = 1 + L(t_i,t_i+1)*(t_i+1-t_i)
  # r_t_i  = np.log(1 + L(t_i,t_i+1)*(t_i+1-t_i))/(t_i+1-t-i)
  # L(t_i,t_i+1) = 
  # LIBOR rates vs counterparty value and LIBOR rates vs stock prices are uncorrelated

# Initialize Forward rates Fj from existing ZCB bond prices using Fj(t_0) = (Pj(0)-Pj+1(0)) / (delta_j*Pj+1(0)) delta_j = Tj+1 - Tj
mc_forward                 = np.ones([num_simulations,paths])*((ZCB_bond_prices[:-1]-ZCB_bond_prices[1:])/(2*ZCB_bond_prices[1:]))

def simulate_rates(libor_rate):
    return np.log(1+ libor_rate)

In [421]:
# 10 - Initialization of variables to be simulated

# Sample paths
r_sim                     = np.zeros([num_simulations, paths+1])   # various simulation paths for interest rates
S_sim                     = np.zeros([num_simulations, paths+1])   # various simulation paths for stock prices
V_sim                     = np.zeros([num_simulations, paths+1])   # various simulation paths for firm value
sigma_share_sim           = np.zeros([num_simulations, paths+1])   # various simulation paths for stock volatility
sigma_firm_sim            = np.zeros([num_simulations, paths+1])   # various simulation paths for firm volatility

#Initial Values
S_sim[:,0]                = S_0                                    # initial stock prices 
V_sim[:,0]                = V_0                                    # initial firm value
sigma_firm_sim[:,0]       = 0.3*(S_0**(gamma-1))                   # initial firm volatility
sigma_share_sim[:,0]      = 0.3*(V_0**(gamma-1))                   # initial firm volatility

In [422]:
# 11 - Implementation of Monte Carlo Simulation to price option with no risk of default

# Z random variables
Z1                        = norm.rvs(size = [num_simulations,paths+1])

for i in range(paths): 
    r_sim[:,i]            = simulate_rates(mc_forward[:,i])
    S_sim[:,i+1]          = next_share_price(S_sim[:,i],r_sim[:,i], sigma_share_sim[:,i], Z1[:,i],T[i])
    
    sigma_share_sim[:,i+1]= next_share_volatility(sigma_share_sim[:,i],S_sim[:,i],gamma)
    sigma_firm_sim[:,i+1] = next_firm_volatility(sigma_firm_sim[:,i],V_sim[:,i],gamma)

# Valuation without risk of default
discounted_payoffs        = discounted_option_payoff(S_sim,r_sim,K,L,T)   
call_value_no_default     = np.mean(discounted_payoffs)
std_no_default            = np.std(discounted_payoffs)/np.sqrt(num_simulations) 
(call_value_no_default,std_no_default)

C:\Users\Zed\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in power
  # Remove the CWD from sys.path while we load stuff.


(5.6312098529099135, 0.02075384303337478)

In [426]:
# 12 - Implementation of Monte Carlo Simulation to price option with risk of default

# Take into account correlation between firm value and stock price 
Z1                        = norm.rvs(size = [num_simulations,paths+1])
Z2                        = norm.rvs(size = [num_simulations,paths+1])
Z2                        =  corr*Z1 + np.sqrt(1-corr**2)*Z2   #creating correlated standard normals

for i in range(paths):  
    r_sim[:,i+1]          = simulate_rates(mc_forward[:,i])
    S_sim[:,i+1]          = next_share_price(S_sim[:,i],r_sim[:,i], sigma_share_sim[:,i], Z1[:,i],T[i])
    
    V_sim[:,i+1]          = next_firm_value(V_sim[:,i],r_sim[:,i], sigma_firm_sim[:,i], Z2[:,i],T[i])
    sigma_share_sim[:,i+1]= next_share_volatility(sigma_share_sim[:,i],S_sim[:,i],gamma)
    sigma_firm_sim[:,i+1] = next_firm_volatility(sigma_firm_sim[:,i],V_sim[:,i],gamma)

# Valuation with risk of default
discounted_payoffs        = discounted_option_payoff(S_sim,r_sim,K,L,T)   
call_value                = np.mean(discounted_payoffs)
amount_lost               = credit_value_adjustment(r_sim,r_r,V_sim,D,call_value)

cva_estimates             = np.mean(amount_lost)
std_estimates             = np.std(amount_lost)/np.sqrt(num_simulations)

call_value_with_default        = value_no_default - cva_estimates
call_value_with_default

5.4346480929373975